In [17]:
from openai import OpenAI
from pydantic import BaseModel, Field, TypeAdapter

In [18]:
class Query(BaseModel):
    """Input format to query the my_function tool"""
    input_value: float = Field(..., description="The input value to the function")

def my_function(query:Query) -> float:
    return query.input_value**2

In [19]:
my_function.__annotations__['query'].model_json_schema()

{'description': 'Input format to query the my_function tool',
 'properties': {'input_value': {'description': 'The input value to the function',
   'title': 'Input Value',
   'type': 'number'}},
 'required': ['input_value'],
 'title': 'Query',
 'type': 'object'}

In [23]:
from typing import Callable

class LLMWithTools:
    def __init__(self, tools:list[Callable]):
        self.client = OpenAI()
        self.tools = []
        self.tool_names = {t.__name__:t for t in tools}
        for tool in tools:
            self.tools.append(
                {
                    "type": "function",
                    "function": {
                        "name": tool.__name__,
                        "description": tool.__doc__,
                        "parameters": {"additionalProperties": False, **tool.__annotations__['query'].model_json_schema()},
                        },
                    "strict": True
                }
            )

    def invoke(self, prompt:str) -> str:
        state = [{"role": "user", "content": prompt}]

        def _client_call():
            return self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=state,
                tools=self.tools
            )
        
        response = _client_call()

        if response.choices[0].message.tool_calls is not None and len(response.choices[0].message.tool_calls) > 0:
            state.append(response.choices[0].message)
            while response.choices[0].message.tool_calls is not None and len(response.choices[0].message.tool_calls) > 0:
                for tool_call in response.choices[0].message.tool_calls:
                    tool = self.tool_names[tool_call.function.name]
                    query = TypeAdapter(tool.__annotations__['query']).validate_json(tool_call.function.arguments)
                    out = tool(query)
                    state.append({
                        "role": "tool",
                          "content": f'{out}',
                          "tool_call_id": tool_call.id,
                          })
                response = _client_call()

        return response.choices[0].message.content

In [24]:
llm = LLMWithTools([my_function])

In [25]:
response = llm.invoke("What is the value of my_function(3)?")

In [27]:
response

'The value of `my_function(3)` is 9.0.'